In [1]:
import sys
import numpy as np
import pandas as pd

#from IPython.core.debugger import set_trace

In [2]:
from dmbtools.wellbore.survey import position_log
from dmbtools.wellbore.horizontal_stats_pak import h_stats, h_stats_meta
from dmbtools.geodetic.projection import GridConvergence, Project, get_epsg_name
from dmbtools.geodetic import convert as cvt
from dmbtools.geodetic import projection as prj
from dmbtools.math import utils as mutils

In [3]:
oneuwi = '429991861452'
epgs_geo_27 = 4267
epsg_crs_27 = 32039
spcs_27 = 'SPCS27 - Texas Central'
epgs_geo_83 = 4269
epsg_crs_83 = 32139
spcs_83 = 'SPCS83 - Texas Central'
latlon = [31.08137, -103.46939]

In [4]:
# 31 43 21.85539(N) 103 49 14.83427(W)
#                     North         East     Units Scale Factor Converg.
# SPC TX C     - 3,233,113.731   369,538.808   MT  0.99996081   -1 47 46.5
lat_CC0554 = cvt.dms2decdeg(31, 43, 21.85539)
lon_CC0554 = cvt.dms2decdeg(-103, 49, 14.83427)
con_CC0554 = cvt.dms2decdeg(-1, 47, 46.5)

In [5]:
gc = GridConvergence(4269, 32139)

In [6]:
gc._prj._from_proj

Proj('+proj=longlat +datum=NAD83 +no_defs', preserve_units=True)

In [7]:
gc._prj._to_proj

Proj('+proj=lcc +lat_0=29.6666666666667 +lon_0=-100.333333333333 +lat_1=31.8833333333333 +lat_2=30.1166666666667 +x_0=700000 +y_0=3000000 +datum=NAD83 +units=m +no_defs', preserve_units=True)

In [8]:
gc.calc(np.array([[lat_CC0554, lon_CC0554]]))

array([-1.79624414])

In [9]:
con_CC0554

-1.7962500000000001

In [10]:
cvt.decdeg2dms(gc.calc(np.array([[lat_CC0554, lon_CC0554]])))

(array([-1.]), array([47.]), array([46.47890342]))

In [11]:
cvt.get_fips_api_spc_epsg_utm_cross_ref()

STATE_NAME STATE_ABBREVIATION COUNTY_NAME    TYPE   LATITUDE   LONGITUDE  \
0       Alabama                 AL     Autauga  County  32.542207  -86.641472   
1       Alabama                 AL     Baldwin  County  30.654881  -87.754736   
2       Alabama                 AL     Barbour  County  31.863073  -85.388993   
3       Alabama                 AL        Bibb  County  32.996943  -87.126855   
4       Alabama                 AL      Blount  County  33.979740  -86.566214   
...         ...                ...         ...     ...        ...         ...   
3094    Wyoming                 WY  Sweetwater  County  41.657588 -108.894093   
3095    Wyoming                 WY       Teton  County  43.922545 -110.573554   
3096    Wyoming                 WY       Uinta  County  41.285990 -110.546505   
3097    Wyoming                 WY    Washakie  County  43.907220 -107.684285   
3098    Wyoming                 WY      Weston  County  43.835646 -104.563095   

     STFIPS COFIPS   FIPS STAPI COAPI UTM SPC27 SPC83  EPSG_NAD27  EPSG_NAD83  
0        01    001  01001    01   001  16  0102  0102       26730       26930  
1        01    003  01003    01   003  16  0102  0102       26730       26930  
2        01    005  01005    01   005  16  0101  0101       26729       26929  
3        01    007  01007    01   007  16  0102  0102       26730       26930  
4        01    009  01009    01   009  16  0102  0102       26730       26930  
...     ...    ...    ...   ...   ...  ..   ...   ...         ...         ...  
3094     56    037  56037    49   037  12  4903  4903       32057       32157  
3095     56    039  56039    49   039  12  4904  4904       32058       32158  
3096     56    041  56041    49   041  12  4904  4904       32058       32158  
3097     56    043  56043    49   043  13  4902  4902       32056       32156  
3098     56    045  56045    49   045  13  4901  4901       32055       32155  

[3099 rows x 16 columns]

In [12]:
cvt.api_num_2_epsg(['300150011700'])

API
300150011700    32012
Name: EPSG_CODE, dtype: int64

In [13]:
codes = cvt.get_fips_api_spc_epsg_utm_cross_ref()[['STAPI', 'COAPI', 'EPSG_NAD27']]
codes = codes.rename(index=str, columns={'EPSG_NAD27': 'EPSG_CODE'})
codes

STAPI COAPI  EPSG_CODE
0       01   001      26730
1       01   003      26730
2       01   005      26729
3       01   007      26730
4       01   009      26730
...    ...   ...        ...
3094    49   037      32057
3095    49   039      32058
3096    49   041      32058
3097    49   043      32056
3098    49   045      32055

[3099 rows x 3 columns]

In [14]:
apinums = [oneuwi]

In [15]:
parsed = [[api, api[:2], api[2:5]] for api in apinums]
cols = ['API', 'STAPI', 'COAPI']
#types = {fld: object for fld in cols}
apis = pd.DataFrame(parsed, columns=cols, dtype=str)
apis

API STAPI COAPI
0  429991861452    42   999

In [16]:
codes[codes['COAPI'] == '999']

Empty DataFrame
Columns: [STAPI, COAPI, EPSG_CODE]
Index: []

In [17]:
prt = Project(4269, 32139)

In [18]:
str(prt)

'from: \n<Geographic 2D CRS: EPSG:4269>\nName: NAD83\nAxis Info [ellipsoidal]:\n- Lat[north]: Geodetic latitude (degree)\n- Lon[east]: Geodetic longitude (degree)\nArea of Use:\n- name: North America - NAD83\n- bounds: (167.65, 14.92, -47.74, 86.46)\nDatum: North American Datum 1983\n- Ellipsoid: GRS 1980\n- Prime Meridian: Greenwich\n \nto: \n<Projected CRS: EPSG:32139>\nName: NAD83 / Texas Central\nAxis Info [cartesian]:\n- X[east]: Easting (metre)\n- Y[north]: Northing (metre)\nArea of Use:\n- name: USA - Texas - SPCS - C\n- bounds: (-106.66, 29.78, -93.5, 32.27)\nCoordinate Operation:\n- name: SPCS83 Texas Central zone (meters)\n- method: Lambert Conic Conformal (2SP)\nDatum: North American Datum 1983\n- Ellipsoid: GRS 1980\n- Prime Meridian: Greenwich\n'

In [19]:
cvt.wai()

C:\ProgramData\Anaconda3\lib\site-packages\dmbtools\resources\fips_api_spc_epsg_utm_cross_ref.csv


In [20]:
cvt.test()

testing 1,2,2...


In [21]:
type(prt._to_proj)

pyproj.proj.Proj

In [22]:
prt._to_proj.definition_string()

'proj=lcc lat_0=29.6666666666667 lon_0=-100.333333333333 lat_1=31.8833333333333 lat_2=30.1166666666667 x_0=700000 y_0=3000000 datum=NAD83 units=m no_defs ellps=GRS80 towgs84=0,0,0'

In [23]:
prt._to_proj.crs

<Projected CRS: EPSG:32139>
Name: NAD83 / Texas Central
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA - Texas - SPCS - C
- bounds: (-106.66, 29.78, -93.5, 32.27)
Coordinate Operation:
- name: SPCS83 Texas Central zone (meters)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [24]:
prt._from_proj.crs.name

'NAD83'

In [25]:
type(prt._to_proj.srs)

str

In [26]:
print(prt)

from: 
<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich
 
to: 
<Projected CRS: EPSG:32139>
Name: NAD83 / Texas Central
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA - Texas - SPCS - C
- bounds: (-106.66, 29.78, -93.5, 32.27)
Coordinate Operation:
- name: SPCS83 Texas Central zone (meters)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich



In [27]:
get_epsg_name(epsg_crs_27)

'NAD27 / Texas Central'

In [28]:
get_epsg_names(epsg_crs_27)

NameError: name 'get_epsg_names' is not defined